In [36]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random
import torch.nn as nn

SEED = 1234
MAX_VOCAB_SIZE = 1_000
BATCH_SIZE = 64

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(vars(train_data.examples[0]))

train_data, valid_data = train_data.split(random_state = random.seed(SEED))
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
print(TEXT.vocab.freqs.most_common(20))
print(LABEL.vocab.stoi)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

Number of training examples: 25000
Number of testing examples: 25000
{'text': ['For', 'a', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'a', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem', '.', 'Imagine', 'a', 'movie', 'where', 'Joe', 'Piscopo', 'is', 'actually', 'funny', '!', 'Maureen', 'Stapleton', 'is', 'a', 'scene', 'stealer', '.', 'The', 'Moroni', 'character', 'is', 'an', 'absolute', 'scream', '.', 'Watch', 'for', 'Alan', '"', 'The', 'Skipper', '"', 'Hale', 'jr', '.', 'as', 'a', 'police', 'Sgt', '.'], 'label': 'pos'}
Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000
Unique tokens in TEXT vocabulary: 1002
Unique tokens in LABEL vocabulary: 2
[('the', 202478), (',', 192130), ('.', 165401), ('a', 109230), ('and', 109175), ('of', 101087), ('to', 93504), ('is', 76398), ('in', 61293), ('I', 54008), ('it', 53329), ('that', 48904), ('"', 44045), ("'s", 43248), ('this', 42372), ('-', 37498), ('/

In [49]:
class Net(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)        
        self.fc_0 = nn.Linear(embedding_dim, hidden_dim)        
        self.fc_1 = nn.Linear(hidden_dim, output_dim)
        self.act = nn.ReLU()
    def forward(self, text):

        #text = [sent len, batch size]
        embedded = self.embedding(text)
        len_sen = embedded.size()[0]
        embedded = self.act(embedded)
        embedded=torch.sum(embedded,dim=0)/ len_sen
        #embedded = [sent len, batch size, emb dim]
        
        output = self.fc_0(embedded)
        output = self.act(output)
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        output = self.fc_1(output)
        
        return output

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 16
OUTPUT_DIM = 2

model = Net(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
print(f'The model has {count_parameters(model):,} trainable parameters')
print(INPUT_DIM)

The model has 101,850 trainable parameters
1002
The model has 101,850 trainable parameters


In [58]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())
# optimizer = optim.SGD(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)


def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
  #  rounded_preds = torch.round(torch.sigmoid(preds))
    rounded_preds = torch.max(preds, 1)[1]
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [59]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        if batch.text.size()[1] != batch.label.size()[0]:
            print(batch.text.size()) 
            print(batch.label.size())   
        optimizer.zero_grad()
        # print(batch.text.size()) 
        # print(batch.label.size())
        predictions = model(batch.text).squeeze(1)
 #       predictions = torch.sum(predictions,dim=0) / predictions.size()[0]
        loss = criterion(predictions, batch.label.long())
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            # predictions = torch.sum(predictions,dim=0) / predictions.size()[0]
            loss = criterion(predictions, batch.label.long())
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 5

best_valid_loss = float('inf')
print(model)
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Net(
  (embedding): Embedding(1002, 100)
  (fc_0): Linear(in_features=100, out_features=16, bias=True)
  (fc_1): Linear(in_features=16, out_features=2, bias=True)
  (act): ReLU()
)
Epoch: 01 | Epoch Time: 0m 10s
	Train Loss: 0.693 | Train Acc: 50.54%
	 Val. Loss: 0.690 |  Val. Acc: 51.26%
Epoch: 02 | Epoch Time: 0m 10s
	Train Loss: 0.691 | Train Acc: 53.75%
	 Val. Loss: 0.685 |  Val. Acc: 56.51%
Epoch: 03 | Epoch Time: 0m 10s
	Train Loss: 0.683 | Train Acc: 57.15%
	 Val. Loss: 0.641 |  Val. Acc: 64.48%
Epoch: 04 | Epoch Time: 0m 10s
	Train Loss: 0.660 | Train Acc: 63.37%
	 Val. Loss: 0.621 |  Val. Acc: 66.61%
Epoch: 05 | Epoch Time: 0m 11s
	Train Loss: 0.622 | Train Acc: 68.25%
	 Val. Loss: 0.659 |  Val. Acc: 68.47%


In [60]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.623 | Test Acc: 65.89%
